In [19]:
pip install pdfquery


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
from pdfquery import PDFQuery
import re

pdf = PDFQuery('../data/Mes comptes _ BNP Paribas.pdf')
pdf.load()

# Use CSS-like selectors to locate the elements
text_elements = pdf.pq('LTTextLineHorizontal')

# Extract the text from the elements
text = [t.text for t in text_elements]

print(text)

['', '', '', '', '', '', '', '', '', 'COMPTE DE CHÈQUES ', '\ue633 N° ****9641 ', 'Facilité de caisse : 100,00 € ', '', '', '4.977,91 € ', 'Prévisionnel : 4.977,91 € ', '', '', '', '', "COMPTE D'ÉPARGNE ", '\ue633 N° ****3775 ', '', '', '64.971,07 € ', '| Prévisionnel : 64.971,07 € ', 'LIVRET DÉV. DURABLE ET SOLIDAIRE ', '\ue633 N° ****1024 ', '', '', '12.655,71 € ', '| Prévisionnel : 12.655,71 € ', 'LIVRET JEUNE ', '\ue633 N° ****2649 ', '', '', '2.127,54 € ', '| Prévisionnel : 2.127,54 € ', '', '', 'Citadine, SUV compact ou familial, proﬁtez des oﬀres du moment pour changer de ', 'voiture. ', 'Trouvez le véhicule qui vous convient sur Mobility4You. ', '#BNPParibasMobility - Pensez à covoiturer. ', '', 'Bonjour Mlle FYOT TAMADON ', 'Dernière connexion le 15/03/2025 à 10h44 ', 'Avoirs 84.732,23 € ', '', '', '', '', '', '', '', '', '', 'Vous n’avez pas de crédit. Besoin de ﬁnancer un ', 'projet ? ', '', 'Découvrir nos ', 'crédits ', '']


In [21]:
clean_text = text
clean_text = [t for t in clean_text if t != '']
clean_text = [t for t in clean_text if "\ue633" not in t]
clean_text = [t for t in clean_text if "Facilité de caisse" not in t]
clean_text = [t for t in clean_text if "Prévisionnel" not in t]
clean_text = [item.strip() for item in clean_text if re.search(r"[\d,.]+ €", item) or "COMPTE" in item or "LIVRET" in item]

print(clean_text)

['COMPTE DE CHÈQUES', '4.977,91 €', "COMPTE D'ÉPARGNE", '64.971,07 €', 'LIVRET DÉV. DURABLE ET SOLIDAIRE', '12.655,71 €', 'LIVRET JEUNE', '2.127,54 €', 'Avoirs 84.732,23 €']


In [22]:
def get_next_value(target_list, key):
    """
    Récupère la valeur immédiatement après un mot-clé ou dans la même chaîne.
    """
    for i in range(len(target_list) - 1):  
        if key in target_list[i]:
            # Si le montant est sur la ligne suivante, on le retourne directement
            return float(target_list[i + 1].replace('.', '').replace(',', '.').replace(' €', ''))
    
    # 🔍 Vérification si la valeur est dans la même ligne (ex: "Avoirs 84.732,23 €")
    for item in target_list:
        if key in item:
            match = re.search(r"[\d,.]+ €", item)  # Extraction du montant dans la même chaîne
            return float(match.group(0).replace('.', '').replace(',', '.').replace(' €', '')) if match else None

    return None  # Si non trouvé

# Tester la correction
compte_cheque = get_next_value(clean_text, "COMPTE DE CHÈQUES")
compte_epargne = get_next_value(clean_text, "COMPTE D'ÉPARGNE")
livret_dev_durable = get_next_value(clean_text, "LIVRET DÉV. DURABLE ET SOLIDAIRE")
livret_jeune = get_next_value(clean_text, "LIVRET JEUNE")
avoirs = get_next_value(clean_text, "Avoirs")

# Affichage des résultats mis à jour
comptes = {
    "compte_cheque": compte_cheque,
    "compte_epargne": compte_epargne,
    "livret_dev_durable": livret_dev_durable,
    "livret_jeune": livret_jeune,
    "avoirs": avoirs
}


In [23]:
import json 
with open("../data/comptes.json", "w", encoding="utf-8") as fichier:
    json.dump(comptes, fichier, ensure_ascii=False, indent=4)